In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *


In [2]:
E, nu = 50, 0.1
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)


In [3]:
L = 1
w = 0.2
left = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
right = Plane(Pnt(L,0,0), Vec(1,0,0)).bc("right")
bottom = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("bottom")
top = Plane(Pnt(0,w,0), Vec(0,1,0)).bc("top")
front = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("front")
back = Plane(Pnt(0,0,w), Vec(0,0,1)).bc("back")
cube = left * right * bottom * top * front * back

geo = CSGeometry()
geo.Add(cube)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))



In [4]:
import scipy as sp
import numpy as np
def Solve_schur(A,B,C,L1,L2,res = False):
        """
        All in ngsolve format and assembled
        """
    
        A_rows,A_cols,A_vals = A.mat.COO()
        B_rows,B_cols,B_vals = B.mat.COO()
        C_rows,C_cols,C_vals = C.mat.COO()

        # get sparse matrix
        A_mat_full = sp.sparse.csr_matrix((A_vals, (A_rows, A_cols)), shape=A.mat.shape)
        fd = np.nonzero(A.space.FreeDofs())[0]

        A_mat = A_mat_full[fd,:][:,fd]
        A_inv = sp.sparse.linalg.inv(A_mat)
        B_mat_full = sp.sparse.csr_matrix((B_vals, (B_rows, B_cols)), shape=B.mat.shape)
        # only take the freedofs in the rows of B
        B_mat = B_mat_full[fd,:]

        C_mat = sp.sparse.csr_matrix((C_vals, (C_rows, C_cols)), shape=C.mat.shape)
        C_inv = sp.sparse.linalg.inv(C_mat)
        

        # get vectors in numpy form
        L1_vec_full = L1.vec.FV().NumPy()
        L1_vec = L1_vec_full[fd]
        if res:
            res_n_vec = res.vec.FV().NumPy()[fd]
        

        L2_vec = L2.vec.FV().NumPy()

        # schur complement for u_it
        F = B_mat @ C_inv @ B_mat.T
        # get schur complement inverse
        # schur_u = A_inv - F
        schur_u_inv = sp.sparse.linalg.inv(A_mat - F)

        schur_delta_inv = sp.sparse.linalg.inv(C_mat - B_mat.T @ A_inv @ B_mat)
        u_vec = np.zeros(len(L1_vec_full))
        u_vec[fd] = schur_u_inv @ (L1_vec- B_mat @ C_inv @ L2_vec) # solve for u_it
        delta_vec = schur_delta_inv @ (L2_vec - B_mat.T @ A_inv @ (L1_vec)) # solve for delta
        
        return u_vec, delta_vec

In [5]:
# H1-conforming finite element space
fesu = VectorH1(mesh, order=3, dirichlet='left')
fesphi = L2(mesh, order=0)

# define trial- and test-functions
u, v= fesu.TnT()
delta, vphi = fesphi.TnT()
gfu_u = GridFunction(fesu)

gfdelta = GridFunction(fesphi)

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 
obs = CF(-0.1)    

max_PG_it = 20
max_iter_newton = 30
rho = 0.1
grav = 9.81
force = CoefficientFunction((0,0,-rho*grav))
alpha = Parameter(1)

for k in range(1,max_PG_it):

    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psih.vec
    for i in range(1,max_iter_newton):
        print("Newton iteration: ", i)
        a = BilinearForm(fesu)
        b = BilinearForm(trialspace=fesphi, testspace=fesu)
        c = BilinearForm(fesphi)
        L1 = LinearForm(fesu)
        L2 = LinearForm(fesphi)

        a += alpha * InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx
        # proximal galerkin term
        b += InnerProduct(delta,v[2])*dx
        # the left hand side
        c+= -delta*exp(psih)*vphi*dx
        c+= -1e-6*delta*vphi*dx
        # the right hand side
        L1 += alpha*force*v*dx
        L1 += (psik-psih)*v[2]*dx

        L2 += (obs+exp(psih))*vphi*dx
        a.Assemble()
        b.Assemble()
        c.Assemble()
        L1.Assemble()
        L2.Assemble()
        print("Attempting schur solve")
        u_np , delta_np =Solve_schur(a,b,c,L1,L2)
        print(sp.linalg.norm(delta_np))
        if sp.linalg.norm(delta_np) < 1e-5:
            print("Converged")
            break
        print("Schur solve finished")
        # update the gridfunctions
        gfu_u.vec.FV().NumPy()[:] = u_np
        gfdelta.vec.FV().NumPy()[:] = delta_np

        psih.vec.data = psih.vec + gfdelta.vec


    Draw(gfu_u, mesh, "u")
    # r.data = lf.vec - a.mat * gfu.vec
    #gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * lf.vec
    if Integrate((gfu_u-uk)**2, mesh) < 1e-5:
        print("Tol PG")
        break
    



Iteration of proximal galerkin:  1
Newton iteration:  1
Attempting schur solve


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


7.465515620939359
Schur solve finished
Newton iteration:  2
Attempting schur solve
6.489196842829828
Schur solve finished
Newton iteration:  3
Attempting schur solve
3.7936468952984326
Schur solve finished
Newton iteration:  4
Attempting schur solve
1.6109141796298685
Schur solve finished
Newton iteration:  5
Attempting schur solve
0.4074321420559382
Schur solve finished
Newton iteration:  6
Attempting schur solve
0.029642770771786727
Schur solve finished
Newton iteration:  7
Attempting schur solve
0.000179297140775137
Schur solve finished
Newton iteration:  8
Attempting schur solve
9.374227274752312e-09
Converged


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  2
Newton iteration:  1
Attempting schur solve
11.456733350011277
Schur solve finished
Newton iteration:  2
Attempting schur solve
10.600645018967711
Schur solve finished
Newton iteration:  3
Attempting schur solve
7.404572576190119
Schur solve finished
Newton iteration:  4
Attempting schur solve
19.149636216328016
Schur solve finished
Newton iteration:  5
Attempting schur solve
4.044590613812635
Schur solve finished
Newton iteration:  6
Attempting schur solve
1.6441032526748969
Schur solve finished
Newton iteration:  7
Attempting schur solve
0.2896921187070122
Schur solve finished
Newton iteration:  8
Attempting schur solve
0.011491135835586187
Schur solve finished
Newton iteration:  9
Attempting schur solve
7.518867151557197e-05
Schur solve finished
Newton iteration:  10
Attempting schur solve
2.3593025981025345e-06
Converged


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  3
Newton iteration:  1
Attempting schur solve
157.38615623544203
Schur solve finished
Newton iteration:  2
Attempting schur solve
5.49042918770978
Schur solve finished
Newton iteration:  3
Attempting schur solve
1.0582532153449447
Schur solve finished
Newton iteration:  4
Attempting schur solve
0.06101303454003092
Schur solve finished
Newton iteration:  5
Attempting schur solve
0.002857922775979933
Schur solve finished
Newton iteration:  6
Attempting schur solve
0.00021329790694678125
Schur solve finished
Newton iteration:  7
Attempting schur solve
1.6096714608343416e-05
Schur solve finished
Newton iteration:  8
Attempting schur solve
1.2148153606577031e-06
Converged


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Tol PG


In [8]:
gfu_tilde = GridFunction(fesu)
gfu_tilde.vec.data = gfu_u.vec
gfu_tilde.components[2].Set(obs + exp(psih))
Draw(gfu_tilde, mesh, "u")  

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [7]:
plot_psi = GridFunction(H1(mesh, order=1))
plot_psi.Set(gfu.components[1])
Draw(plot_psi, mesh, "psi")

NameError: name 'gfu' is not defined

In [7]:
gfu_u.vec.data = gfu.components[0].vec
gfu_u.components[2].Set(obs + exp(gfu.components[1]))
Draw(gfu_u, mesh, "u")
Draw(gfu.components[1], mesh, "phi")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [8]:
vec = gfu.components[1]
Draw(vec)
print(min(vec.vec.data))

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

-392.6541068460776


In [9]:
import pickle
with open('gfu.pkl', 'wb') as output:
    pickle.dump(gfu_u, output)
